<h1>진행순서</h1>
<h3 style="margin:0px">
  1. 데이터 전처리 → 2. KoBART 학습 → 3. 요약 모델 실행 (후처리 동시에 진행)
</h3>


<h2>ROUGE 모델 평가 지표</h2>
<h4 style="margin:0px;">
  "eval_loss": 0.13988181948661804, 
  "eval_rouge1": 40.4069, 
  "eval_rouge2": 24.6301, 
  "eval_rougeL": 38.6044, 
  "eval_rougeLsum": 38.6401
</h4>


<h1>데이터 전처리 과정</h1>

데이터 전처리 + KOBART학습 과정 개발환경 -> python 3.12사용 <span style="color:red">(python3.13사용시 라이브러리 호환 문제)</span>
사용 라이브러리
absl-py==2.3.1
accelerate==1.11.0
aiohappyeyeballs==2.6.1
aiohttp==3.13.0
aiosignal==1.4.0
anyio==4.11.0
attrs==25.4.0
certifi==2025.10.5
charset-normalizer==3.4.3
click==8.3.0
colorama==0.4.6
datasets==4.2.0
dill==0.4.0
evaluate==0.4.6
filelock==3.13.1
frozenlist==1.8.0
fsspec==2024.6.1
h11==0.16.0
httpcore==1.0.9
httpx==0.28.1
huggingface-hub==0.35.3
idna==3.10
Jinja2==3.1.4
joblib==1.5.2
MarkupSafe==2.1.5
mpmath==1.3.0
multidict==6.7.0
multiprocess==0.70.16
networkx==3.3
nltk==3.9.2
numpy==2.1.2
packaging==25.0
pandas==2.3.3
pillow==11.0.0
propcache==0.4.1
psutil==7.1.0
pyarrow==21.0.0
python-dateutil==2.9.0.post0
pytz==2025.2
PyYAML==6.0.3
regex==2025.9.18
requests==2.32.5
rouge_score==0.1.2
safetensors==0.6.2
sentencepiece==0.2.1
setuptools==70.2.0
six==1.17.0
sniffio==1.3.1
sympy==1.13.1
tokenizers==0.22.1
torch==2.5.1+cu121
torchaudio==2.5.1+cu121
torchvision==0.20.1+cu121
tqdm==4.67.1
transformers==4.57.1
typing_extensions==4.12.2
tzdata==2025.2
urllib3==2.5.0
xxhash==3.6.0
yarl==1.22.0


In [ ]:
import os, glob, json, re, unicodedata, hashlib
import pandas as pd

INPUT_DIR = r"전처리할 csv파일 경로 입력"
OUTPUT_CSV = r"전처리한 csv파일 저장할 경로 입력"

#보일러플레이트/광고/기자서명 제거
BOILER_PATTERNS = [
    r"\(?사진=.+?\)", r"\[사진\].*?$", r"\[영상\].*?$", r"\[전문\].*?$",
    r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}",  # 이메일
    r"https?://\S+", r"www\.\S+",
    r"[#＃][\w가-힣]+",  # 해시태그
    r"ⓒ.*?$", r"무단전재.*?금지", r"재배포.*?금지", r"※.*?$",
    r"기사제보.*?$", r"네이버.*?구독", r"페이스북.*?팔로우", r"카카오.*?채널",
    r"\([가-힣A-Za-z ]*기자\)", r"[가-힣A-Za-z]+ 기자",  # 기자명
]

def nfkc(s: str) -> str:
    return unicodedata.normalize("NFKC", s or "")

def strip_boiler(s: str) -> str:
    s = s.replace("\r", "\n")
    for pat in BOILER_PATTERNS:
        s = re.sub(pat, " ", s, flags=re.IGNORECASE | re.MULTILINE)
    # 괄호 속 출처/캡션류 간단 제거
    s = re.sub(r"\([^)]{0,40}(출처|자료|사진|영상|그래픽)[^)]{0,40}\)", " ", s)
    return s

def normalize_spaces(s: str) -> str:
    # 연속 공백/탭 축소, 과도한 줄바꿈 축소
    s = re.sub(r"[ \t]+", " ", s)
    s = re.sub(r"\n\s*\n+", "\n\n", s)
    return s.strip()

def ensure_sentence_ending(s: str) -> str:
    # 요약이 문장부호 없이 끝나는 것 보정
    if s and s[-1] not in ".!?。…":
        s += "."
    return s

#정규화 진행. 
def normalize_text(t: str) -> str:
    t = nfkc(t) #전각/반각, 조합문자, 기호 통일
    t = strip_boiler(t) #이메일, URL, 기자명, 저작권문구 등 보일러플레이트 제거
    t = normalize_spaces(t) #연속 공백, 줄바꿈 정리
    return t

# 길이+겹침 기반 스코어, 요약
def jaccard(a_tokens, b_tokens):
    A, B = set(a_tokens), set(b_tokens)
    return len(A & B) / max(1, len(A | B))

WORD_RE = re.compile(r"[가-힣A-Za-z0-9]+")

def tokenize_simple(s: str):
    return WORD_RE.findall(s)

def length_score(n_tokens, low=25, high=120):  # 토큰 기준 가이드
    if n_tokens <= 0: return 0.0
    if n_tokens < low:
        return n_tokens / low  # 짧을수록 점수 감소
    if n_tokens > high:
        # 길수록 감소, high*1.5 지점에서 0.5까지
        return max(0.5, (high*1.5 - n_tokens) / (high*1.5 - high))
    return 1.0

def pick_summary_heuristic(d: dict, passage: str) -> str:
    ann = d.get("Annotation", {})
    cands = [ann.get("summary1"), ann.get("summary2"), ann.get("summary3")]
    cands = [normalize_text(x) for x in cands if x]
    if not cands:
        return ""
    p_tokens = tokenize_simple(passage)
    best, best_score = "", -1.0
    for c in cands:
        c_tokens = tokenize_simple(c)
        ls = length_score(len(c_tokens))
        js = jaccard(set(c_tokens), set(p_tokens))
        score = 0.6 * js + 0.4 * ls  # 가중치(겹침 60%, 길이 40%) — 데이터에 맞게 조절
        if score > best_score:
            best, best_score = c, score
    return ensure_sentence_ending(best.strip())

def extract_row(path: str):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    passage = normalize_text(data.get("Meta(Refine)", {}).get("passage", ""))
    summary = pick_summary_heuristic(data, passage)

    return {
        "article": passage,
        "summary": summary,
        "doc_id": data.get("Meta(Acqusition)", {}).get("doc_id", os.path.basename(path))
    }

def main():
    files = sorted(glob.glob(os.path.join(INPUT_DIR, "*.json")))
    rows = []
    for fp in files:
        try:
            row = extract_row(fp)
            art, summ = row["article"], row["summary"]
            # 길이 필터 (문자 수 기준 + 토큰 비율 기준)
            if len(art) < 80 or len(summ) < 30:
                continue
            a_tok, s_tok = tokenize_simple(art), tokenize_simple(summ)
            if len(s_tok) < 15 or len(a_tok) < 80:
                continue
            ratio = len(a_tok) / max(1, len(s_tok))
            if not (4.0 <= ratio <= 12.0):
                continue
            # 본문-요약 과도한 복붙 제거
            if jaccard(a_tok, s_tok) > 0.85:
                continue
            rows.append(row)
        except Exception as e:
            print(f"[WARN] skip {fp}: {e}")

    df = pd.DataFrame(rows, columns=["doc_id", "article", "summary"])

    # 중복/유사중복 제거
    # 완전중복
    df.drop_duplicates(subset=["article", "summary"], inplace=True)

    # 유사중복(해시 키 기반 빠른 걸러내기)
    def fast_key(s):  # 문장부호 제거 후 해시
        t = re.sub(r"[^\w가-힣]", "", s)
        return hashlib.md5(t.encode("utf-8")).hexdigest()
    df["a_key"] = df["article"].map(fast_key)
    df["s_key"] = df["summary"].map(fast_key)
    df.drop_duplicates(subset=["a_key", "s_key"], inplace=True)
    df.drop(columns=["a_key", "s_key"], inplace=True)

    #저장
    df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print(f"Saved {len(df)} rows to {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


<h1>KOBART학습 과정</h1>

In [ ]:
"""
입력방법 -> 해당 내용을 powershell에 입력
python 파이썬 경로 입력 `
  --train_csv "학습 csv파일 경로 입력" `
  --val_csv   "검증 csv파일 경로 입력" `
  --output_dir "학습시킨 모델 저장할 경로 입력" `
  --------하이퍼 파라미터 값 지정---------
  --epochs 3 `
  --batch_size 2 `
  --grad_accum 2 `
  --max_input_len 1024 `
  --max_target_len 128 `
"""

import argparse
import random
from typing import Dict, Any

import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import evaluate
import torch


def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--train_csv", type=str, required=True,
                    help="학습 CSV (columns: article, summary)")
    ap.add_argument("--val_csv",   type=str, required=True,
                    help="검증 CSV (columns: article, summary)")
    ap.add_argument("--model_id",  type=str, default="gogamza/kobart-summarization")
    ap.add_argument("--output_dir", type=str, default="./kobart-news-sum")
    ap.add_argument("--epochs", type=int, default=1)             # 학습 에폭
    ap.add_argument("--batch_size", type=int, default=2)         # 장치당 배치 크기
    ap.add_argument("--grad_accum", type=int, default=2)         # 그래디언트 누적 단계
    ap.add_argument("--lr", type=float, default=5e-5)
    ap.add_argument("--warmup_ratio", type=float, default=0.03)
    ap.add_argument("--max_input_len", type=int, default=1024)   # 입력 최대 토큰
    ap.add_argument("--max_target_len", type=int, default=128)   # 요약 최대 토큰
    ap.add_argument("--logging_steps", type=int, default=50)
    ap.add_argument("--seed", type=int, default=42)
    args = ap.parse_args()

    set_seed(args.seed)

    # 디바이스/정밀도 자동 설정
    use_cuda = torch.cuda.is_available()
    use_fp16 = False
    use_bf16 = False
    if use_cuda:
        use_fp16 = True   # Ampere(예: RTX 3080) 권장
        use_bf16 = False
        if args.batch_size == 1:
            args.batch_size = 2
        if args.grad_accum > 2:
            args.grad_accum = 2

    # 1) 데이터 로드
    data_files = {"train": args.train_csv, "validation": args.val_csv}
    raw = load_dataset("csv", data_files=data_files)
    dataset_train, dataset_val = raw["train"], raw["validation"]

    # 2) 모델/토크나이저. KoBART 전용 SentencePiece 토크나이저 사용
    tokenizer = AutoTokenizer.from_pretrained(args.model_id, use_fast=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(args.model_id)

    # 요약 태스크에선 분류 라벨 맵 불필요 → 불일치 무력화
    try:
        if hasattr(model.config, "id2label"):
            model.config.num_labels = len(getattr(model.config, "id2label", {})) or 0
    except Exception:
        pass

    # GPU 최적화 옵션
    if use_cuda:
        try:
            model.gradient_checkpointing_enable()   # VRAM 절약
        except Exception:
            pass
        try:
            torch.set_float32_matmul_precision("high")
        except Exception:
            pass

    # 전처리. KoBART의 SentencePiece 기반 토크나이저가 문장을 subword 단위(단어보다 작게, 자주 등장하는 음절/어절) 로 분리하고 라벨을 답니다.
    def preprocess(batch: Dict[str, Any]) -> Dict[str, Any]:
        inputs = batch["article"]
        targets = batch["summary"]

        model_inputs = tokenizer(
            inputs,
            max_length=args.max_input_len,
            truncation=True,
        )

        labels = tokenizer(
            text_target=targets,
            max_length=args.max_target_len,
            truncation=True,
        )["input_ids"]

        model_inputs["labels"] = labels
        return model_inputs

    cols_train = dataset_train.column_names
    cols_val = dataset_val.column_names

    tokenized_train = dataset_train.map(preprocess, batched=True, remove_columns=cols_train)
    tokenized_val   = dataset_val.map(preprocess,   batched=True, remove_columns=cols_val)

    # 데이터 콜레이터 
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        pad_to_multiple_of=8 if use_fp16 else None,
    )

    # 평가 지표 (ROUGE)
    rouge = evaluate.load("rouge")

    def compute_metrics(eval_pred):
        preds, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

        # labels의 -100 → pad token id 로 복구 후 디코딩
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds  = [p.strip() for p in decoded_preds]
        decoded_labels = [l.strip() for l in decoded_labels]
        result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

        # ROUGE 값은 0~1 → %로 변환
        return {k: round(v * 100, 4) if k.startswith("rouge") else v for k, v in result.items()}

    # 반복 억제 기본값
    model.config.no_repeat_ngram_size = 3
    model.config.repetition_penalty = 1.1

    # 학습 설정 
    training_args = Seq2SeqTrainingArguments(
        output_dir=args.output_dir,

        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_rougeL",
        greater_is_better=True,
        save_total_limit=2,

        learning_rate=args.lr,
        warmup_ratio=args.warmup_ratio,
        weight_decay=0.01,

        per_device_train_batch_size=args.batch_size,
        per_device_eval_batch_size=1 if not use_cuda else min(4, args.batch_size * 2),
        gradient_accumulation_steps=args.grad_accum,

        num_train_epochs=args.epochs,
        logging_steps=args.logging_steps,

        predict_with_generate=True,
        generation_max_length=args.max_target_len,

        dataloader_pin_memory=use_cuda,
        fp16=use_fp16,
        bf16=use_bf16,
        report_to="none",

        
        max_grad_norm=1.0,
    )

    # 8) Trainer 
    base_kwargs = dict(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    try:
        trainer = Seq2SeqTrainer(
            processing_class=tokenizer,
            **base_kwargs,
        )
    except TypeError:
        trainer = Seq2SeqTrainer(
            tokenizer=tokenizer,
            **base_kwargs,
        )

    trainer.train()
    trainer.save_model(args.output_dir)
    tokenizer.save_pretrained(args.output_dir)
    print(f"===> Finished. Model saved to: {args.output_dir}")


if __name__ == "__main__":
    main()

<h1>후처리시 사용 함수들 정의</h1>

In [ ]:
import re

def _normalize(text: str) -> str:
    if not text:
        return ""
    t = text.replace("\r", "\n")
    t = re.sub(r"[ \t]+", " ", t)
    t = re.sub(r"\s*\n\s*", "\n", t)
    t = re.sub(r"(\.|\?|\!){2,}", r"\1", t)
    t = re.sub(r"\s*([.,!?…])", r"\1", t)
    t = re.sub(r"([.,!?…])([^\s])", r"\1 \2", t)
    return t.strip()

COMMON_FIXES = [
    (r"\b여 학생\b", "여학생"),
    (r"\b줄어 들\b", "줄어들"),
    (r"\b늘어 날\b", "늘어날"),
    (r"\s*%\s*", "%"),
]

def _apply_common_fixes(text: str) -> str:
    t = text
    for pat, rep in COMMON_FIXES:
        t = re.sub(pat, rep, t)
    return t

ENDINGS_RULES = [
    (r"(로|으로|에|에서|에게|과|와|및|의|를|을|은|는|가|이)[\.\u2026]*$", " 나타났다."),
    (r"(중이다|중|경향이다|경향|추세|상태)[\.\u2026]*$", "이다."),
    (r"(증가|감소|확대|축소|상승|하락)[\.\u2026]*$", "했다."),
    (r"(필요|예정|전망)[\.\u2026]*$", "이다."),
    (r"(한|함)[\.\u2026]*$", " 것으로 보인다."),
]

PHRASE_MAP = {
    "꾸준한.": "꾸준한 증가세를 보였다.",
    "확대.": "확대되는 추세다.",
    "감소.": "감소하는 모습을 보였다.",
}

def _smart_finalize(sentence: str) -> str:
    s = sentence.strip()
    if not s:
        return s
    for k, v in PHRASE_MAP.items():
        if s.endswith(k):
            return s[:-len(k)] + v
    for pat, tail in ENDINGS_RULES:
        if re.search(pat, s):
            s = re.sub(pat, tail, s)
            break
    if s and s[-1] not in ".!?…。":
        s += "."
    return s

def polish_korean_summary(text: str) -> str:
    t = _normalize(text)
    t = _apply_common_fixes(t)
    parts = re.split(r"(?<=[\.!?…。])\s+", t) if t else []
    if parts:
        parts[-1] = _smart_finalize(parts[-1])
    out = " ".join([p for p in parts if p])
    out = _normalize(out)
    return out

__all__ = ["polish_korean_summary"]

<h1>요약 모델 실행 과정</h1>

요약 모델 실행 과정 개발환경 -> python3.13 사용
사용 라이브러리
certifi==2025.10.5
charset-normalizer==3.4.4
colorama==0.4.6
filelock==3.20.0
fsspec==2025.10.0
huggingface-hub==0.36.0
idna==3.11
Jinja2==3.1.6
MarkupSafe==3.0.3
mpmath==1.3.0
networkx==3.5
numpy==2.3.4
packaging==25.0
pandas==2.3.3
python-dateutil==2.9.0.post0
pytz==2025.2
PyYAML==6.0.3
regex==2025.10.23
requests==2.32.5
safetensors==0.6.2
setuptools==80.9.0
six==1.17.0
sympy==1.14.0
tokenizers==0.22.1
torch==2.9.0
tqdm==4.67.1
transformers==4.57.1
typing_extensions==4.15.0
tzdata==2025.2
urllib3==2.5.0

In [ ]:
"""
csv파일 입력 예식
PowerShell에 입력:
& 파이썬 파일 경로 입력 `
  C:실행파일 경로 입력 `
  --in  "입력 csv경로 입력" `
  --out "기사 요약한 csv파일 저장 경로 입력" `
  --text-col article <- 기사 원문이 묶여있는 column입력
"""

#(CUDA 결정론) torch import 전에 설정
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

from pathlib import Path
import json
import re
import random
import numpy as np
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig

# 재현성 고정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cuda.matmul.allow_tf32 = False
    torch.backends.cudnn.allow_tf32 = False
torch.use_deterministic_algorithms(True, warn_only=True)

# (선택) 한국어 후처리 모듈
try:
    from ko_postprocess import polish_korean_summary
except ImportError:
    polish_korean_summary = lambda x: x

# 모델 로드 & 설정 (num_labels/id2label 경고 원천 차단)
MODEL_DIR = Path(r"모델 경로 입력").resolve()

def _clean_config_json_on_disk(model_dir: Path):
    """모델 폴더의 config.json에서 분류 관련 잔여 키 제거 (경고 발생 원천 차단)."""
    cfg_path = model_dir / "config.json"
    if not cfg_path.exists():
        return
    try:
        with open(cfg_path, "r", encoding="utf-8") as f:
            cfg = json.load(f)
    except Exception:
        return
    changed = False
    for k in ["id2label", "label2id", "num_labels", "problem_type"]:
        if k in cfg:
            del cfg[k]
            changed = True
    if changed:
        with open(cfg_path, "w", encoding="utf-8") as f:
            json.dump(cfg, f, ensure_ascii=False, indent=2)

_clean_config_json_on_disk(MODEL_DIR)

tok = AutoTokenizer.from_pretrained(MODEL_DIR.as_posix(), local_files_only=True, use_fast=True)
cfg = AutoConfig.from_pretrained(MODEL_DIR.as_posix(), local_files_only=True)
cfg.id2label, cfg.label2id, cfg.num_labels = {}, {}, 0

model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_DIR.as_posix(),
    local_files_only=True,
    config=cfg,
)
model.config.id2label, model.config.label2id, model.config.num_labels = {}, {}, 0

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device).eval()

def _postprocess(text: str) -> str:
    text = text.strip()
    if text and text[-1] not in ".!?…。\"'”’":
        text += "."
    return text


#가이드/프롬프트
CATEGORY_HINTS = {
    "news_incident": ["사망", "부상", "심장마비", "화재", "사고", "폭발", "체포", "피해", "응급", "이송", "병원"],
    "sports":        ["전반전", "후반전", "득점", "골", "어시스트", "승리", "패배", "라운드", "엘클라시코", "경기"],
    "policy":        ["발표", "정책", "대책", "계획", "공표", "시행", "개정", "대통령", "장관", "국회"],
    "market":        ["실적", "전망", "매출", "영업이익", "주가", "물가", "금리", "환율", "성장률", "지표"],
    "reaction":      ["논란", "비판", "반응", "찬반", "여론", "논의", "논평", "입장", "평가"],
}

COMMON_GUIDE = (
    "※ 아래 글의 핵심 내용을 2~4문장으로 요약하라.\n"
    "※ 핵심 사실(누가, 무엇을, 언제, 어디서, 왜, 어떻게)을 중심으로 정리하라.\n"
    "※ 새 정보는 추가하지 말고 의미를 유지하며 자연스럽게 재구성하라.\n"
)

CATEGORY_GUIDES = {
    "news_incident": "※ 사건의 원인·결과·조치를 중심으로 요약하라.\n",
    "sports": "※ 경기 결과·주요 장면을 요약하라.\n",
    "policy": "※ 정책·발표의 주요 내용을 요약하라.\n",
    "market": "※ 경제 지표나 실적의 변화를 중심으로 요약하라.\n",
    "reaction": "※ 논란·여론의 쟁점을 공정하게 요약하라.\n",
}

def detect_category(text: str) -> str:
    scores = {k: sum(kw in text for kw in kws) for k, kws in CATEGORY_HINTS.items()}
    best = max(scores, key=scores.get)
    return best if scores[best] > 0 else None

def build_guided_input(article: str) -> str:
    cat = detect_category(article)
    return COMMON_GUIDE + CATEGORY_GUIDES.get(cat, "") + article

# 요약 (결정론: 샘플링 X, 빔서치)
def summarize(text: str) -> str:
    inputs = tok(text, return_tensors="pt", truncation=True, max_length=1024)
    # 토큰 id 안전성 체크
    vocab_size = getattr(model.config, "vocab_size", None)
    max_id = int(inputs["input_ids"].max())
    if vocab_size is not None and max_id >= vocab_size:
        raise ValueError(f"Token id {max_id} >= vocab_size {vocab_size}. MODEL_DIR 확인.")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        out = model.generate(
            **inputs,
            do_sample=False,           # 결정론
            num_beams=5,               # 빔서치
            min_new_tokens=40,
            max_new_tokens=150,
            length_penalty=1.0,
            no_repeat_ngram_size=4,
            repetition_penalty=1.2,
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.pad_token_id,
        )
    return _postprocess(tok.decode(out[0], skip_special_tokens=True))

# 후처리 (비문/중복 정리 -> 문장수 제한 순서!)
def _norm_sent(s: str) -> str:
    s = re.sub(r"\s+", " ", s)
    s = s.replace("'", "").replace('"', "")
    s = re.sub(r"[‘’“”]", "", s)
    return s.strip().lower()

def dedupe_similar_sentences(text: str, sim_threshold: float = 0.8) -> str:
    sents = [s for s in re.split(r"(?<=[\.!?…。])\s+", text.strip()) if s]
    kept, norm_tokens = [], []

    def tokens(x): return set(_norm_sent(x).split())

    for s in sents:
        tk = tokens(s)
        if not tk:
            continue
        dup = False
        for prev_tk in norm_tokens:
            inter = len(tk & prev_tk)
            union = len(tk | prev_tk) or 1
            jacc = inter / union
            if jacc >= sim_threshold:
                dup = True
                break
        if not dup:
            kept.append(s)
            norm_tokens.append(tk)
    return " ".join(kept).strip()

def tidy_korean_summary(text: str) -> str:
    t = text
    t = t.replace("||", " ")
    t = re.sub(r"[‘’]", "'", t)
    t = re.sub(r'[“”]', '"', t)
    t = re.sub(r"\s+", " ", t).strip()

    # [출처] 태그 중복 제거
    m = re.match(r"^\[([^\[\]]{1,30})\]\s*", t)
    if m:
        tag = m.group(0)
        body = t[len(tag):]
        body = re.sub(r"\[\s*"+re.escape(m.group(1))+r"\s*\]\s*", "", body)
        t = (tag + body).strip()

    # 마침표/따옴표 정리
    t = re.sub(r"\.\s*\.", ".", t)
    t = re.sub(r'"\s*"', '"', t)

    # 문장 분리
    sents = [s.strip() for s in re.split(r"(?<=[\.!?…。])\s+", t) if s.strip()]

    #비문/토막/명사형 종결 제거 
    BAD_ENDINGS = (
        "은.", "는.", "이.", "가.", "를.", "을.", "로.", "과.", "와.", "에.", "의.",
        "인.",  # ← 문제였던 케이스
        "중.", "등.", "및.", "또.", "며.", "듯.", "만.", "뿐."
    )
    # 한국어 서술 종결(대략) 판정: ‘…다.’ 계열/의문·감탄/따옴표로 끝나는 문장 허용
    VERB_END = re.compile(r"(다|했다|됐다|였다|잇다|있다|없다|밝혔다|전했다|강조했다|열렸다|발표했다|추진한다)\.$")

    clean = []
    for s in sents:
        s_norm = re.sub(r"\s+", "", s)

        # 너무 짧은 문장 제거
        if len(s_norm) < 8:
            continue

        # 조사·명사형으로 끝나는 문장 제거
        if any(s_norm.endswith(be) for be in BAD_ENDINGS):
            continue

        # 서술형 종결이 아닌 경우(명사형 종결 추정) 제거
        if not (VERB_END.search(s) or s_norm.endswith(("?", "!", "…."))):
            continue

        clean.append(s)

    # 마지막 문장 재검사(혹시 남아있으면 한 번 더 컷)
    if clean and any(clean[-1].strip().endswith(be) for be in BAD_ENDINGS):
        clean = clean[:-1]

    t = " ".join(clean).strip()

    # 따옴표 짝수 보정
    if t.count('"') % 2 == 1:
        t = t.replace('"', '')
    if t.count("'") % 2 == 1:
        t = t.replace("'", "")

    if t and t[-1] not in ".!?…。\"'”’":
        t += "."

    return t


def minimal_cleanup(summary: str, keep: int = 4) -> str:
    """마지막에 문장수만 제한."""
    text = summary.strip()
    sents = [s for s in re.split(r"(?<=[\.!?…。])\s+", text) if s]
    text = " ".join(sents[:keep]).strip()
    return text

def summarize_article(article: str, *, keep_sents: int = 4) -> str:
    guided = build_guided_input(article)
    raw = summarize(guided)
    cleaned = tidy_korean_summary(raw)                      # 1) 정리
    cleaned = dedupe_similar_sentences(cleaned, 0.8)        # 2) 중복 제거
    cleaned = minimal_cleanup(cleaned, keep=keep_sents)     # 3) 문장 수 제한
    try:
        cleaned = polish_korean_summary(cleaned)            # 4) (선택) 후광택
    except Exception:
        pass
    return cleaned

# CSV/XLSX/TSV 일괄 요약
def _read_table_auto(path: str, encoding: str = "utf-8") -> pd.DataFrame:
    ext = os.path.splitext(path)[1].lower()
    if ext in [".xlsx", ".xls"]:
        try:
            return pd.read_excel(path)          # openpyxl 필요
        except Exception:
            return pd.read_excel(path, engine="openpyxl")
    elif ext in [".tsv", ".tab"]:
        return pd.read_csv(path, sep="\t", encoding=encoding)
    else:
        return pd.read_csv(path, encoding=encoding)

def _summarize_series(series, *, keep_sents=4):
    results = []
    series = series.fillna("").astype(str)
    for text in tqdm(series, desc="Summarizing", ncols=100):
        if not text.strip():
            results.append("")
            continue
        try:
            s = summarize_article(text, keep_sents=keep_sents)
        except Exception as e:
            s = f"[ERROR] {type(e).__name__}: {e}"
        results.append(s)
    return results

def summarize_csv(
    input_path: str,
    output_csv: str,
    *,
    text_col: str = "article",
    keep_other_cols: bool = True,
    keep_sents: int = 4,
    encoding: str = "utf-8",
):
    df = _read_table_auto(input_path, encoding=encoding)
    if text_col not in df.columns:
        raise ValueError(f"'{text_col}' 컬럼을 찾을 수 없습니다. 컬럼: {list(df.columns)}")
    summaries = _summarize_series(df[text_col], keep_sents=keep_sents)
    if keep_other_cols:
        df_out = df.copy()
        df_out["summary"] = summaries
    else:
        df_out = pd.DataFrame({"summary": summaries})
    df_out.to_csv(output_csv, index=False, encoding="utf-8-sig")
    return output_csv

# 메인: CLI 또는 단일 데모
if __name__ == "__main__":
    import argparse, sys

    parser = argparse.ArgumentParser(description="KoBART 뉴스 요약 (결정론)")
    parser.add_argument("--in", dest="input_path", type=str, help="입력 경로 (xlsx/csv/tsv)")
    parser.add_argument("--out", dest="output_csv", type=str, help="출력 CSV 경로")
    parser.add_argument("--text-col", dest="text_col", type=str, default="article", help="기사 본문 컬럼명 (기본: article)")
    parser.add_argument("--keep-sents", dest="keep_sents", type=int, default=4, help="최종 문장 수 (기본: 4)")
    parser.add_argument("--encoding", dest="encoding", type=str, default="utf-8", help="입력 텍스트 인코딩(csv/tsv)")
    args = parser.parse_args()

    if args.input_path and args.output_csv:
        saved = summarize_csv(
            args.input_path,
            args.output_csv,
            text_col=args.text_col,
            keep_sents=args.keep_sents,
            encoding=args.encoding,
        )
        print(f"[완료] 저장: {saved}")
        sys.exit(0)

    # 인자 없이 실행되면 단일 데모 (CSV/배치와 동일 파이프라인)
    demo_article = """기사본문 입력"""
    print("\n[요약 결과]\n", summarize_article(demo_article, keep_sents=4))



